In [1]:
# Load the datasets
from datasets import load_multitask_data, SentencePairDataset, SentenceClassificationDataset

In [2]:
sst_train="data/ids-sst-train.csv"
sst_dev="data/ids-sst-dev.csv"
sst_test="data/ids-sst-test-student.csv"
para_train="data/quora-train.csv"
para_dev="data/quora-dev.csv"
para_test="data/quora-test-student.csv"
sts_train="data/sts-train.csv"
sts_dev="data/sts-dev.csv"
sts_test="data/sts-test-student.csv"

In [3]:
class tempArgs():
    def __init__(self,):
        self.local_files_only=True
tempargs = tempArgs()

In [4]:
#Source data. Still need to turn into dataset
sst_train_data, num_labels,para_train_data, sts_train_data = load_multitask_data( sst_train, para_train, sts_train, split ='train')
sst_dev_data, num_labels,para_dev_data, sts_dev_data = load_multitask_data( sst_dev, para_dev, sts_dev, split ='train')
#Data source to dataset
sst_train_data = SentenceClassificationDataset(sst_train_data, tempargs)
sst_dev_data = SentenceClassificationDataset(sst_dev_data, tempargs)

paraphrase_train_data = SentencePairDataset(para_train_data, tempargs, isRegression =False)
paraphrase_dev_data = SentencePairDataset(para_dev_data, tempargs, isRegression =False)

sts_train_data = SentencePairDataset(sts_train_data, tempargs, isRegression =True)
sts_dev_data = SentencePairDataset(sts_dev_data, tempargs, isRegression =True)


Loaded 8544 train examples from data/ids-sst-train.csv
Loaded 141498 train examples from data/quora-train.csv
Loaded 6040 train examples from data/sts-train.csv
Loaded 1101 train examples from data/ids-sst-dev.csv
Loaded 20212 train examples from data/quora-dev.csv
Loaded 863 train examples from data/sts-dev.csv


In [5]:
train_datasets = [sst_train_data,paraphrase_train_data, sts_train_data]

In [6]:
# Load data loader

In [7]:
from data_loader import MultiTaskBatchSampler, MultiTaskDataset

In [8]:
train_datasets = [sst_train_data,paraphrase_train_data, sts_train_data]
mtl_sampler = MultiTaskBatchSampler(        datasets=train_datasets,
        current_epoch=1,
        total_epochs=5,
        batch_size = 128,
        mix_opt=1,
        extra_task_ratio=0,
        bin_size=64,
        bin_on=False,
        bin_grow_ratio=0.5,
        sampling='sequential')

multi_task_train_dataset = MultiTaskDataset(train_datasets)


In [14]:
class CustomCollateFn:
    def __init__(self, collate_fns):
        self.collate_fns = collate_fns

    def __call__(self, batch):
        task_id,_= batch[0] #This tuple is defined in the MultiTaskDataset class
        #This only works if a batch only contains data from one task
        collate_fn = self.collate_fns[task_id]
        actual_batch = [actual_batch for task_id, actual_batch in batch]
        return collate_fn(actual_batch)

# Assuming you have collate functions for each task
collate_fns = {
    0: sst_train_data.collate_fn,
    1: paraphrase_train_data.collate_fn,
    2: sts_train_data.collate_fn
}

In [15]:
# Creating the custom collate function using the dictionary of collate functions
custom_collate_fn = CustomCollateFn(collate_fns)

In [16]:

from torch.utils.data import DataLoader

multi_task_train_data = DataLoader(
    multi_task_train_dataset,
    batch_sampler=mtl_sampler,
    collate_fn = custom_collate_fn
)

In [17]:
i=0
for batch in multi_task_train_data:
    print(batch)
    i=i+1
    if i==1:
        break
    

{'token_ids_1': tensor([[ 101, 2129, 2097,  ...,    0,    0,    0],
        [ 101, 2029, 3152,  ...,    0,    0,    0],
        [ 101, 2054, 2024,  ...,    0,    0,    0],
        ...,
        [ 101, 2054, 2024,  ...,    0,    0,    0],
        [ 101, 6475, 1998,  ...,    0,    0,    0],
        [ 101, 2054, 3633,  ...,    0,    0,    0]]), 'token_type_ids_1': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask_1': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'token_ids_2': tensor([[ 101, 2129, 2097,  ...,    0,    0,    0],
        [ 101, 2054, 2003,  ...,    0,    0,    0],
        [ 101, 2054, 2024,  ...,    0,    0,    0],
        